In [1]:
import sys
import gsi_float_neural_api
import numpy as np
import csv
from datetime import datetime
import h5py

### Allocate Boards

Allocate boards on the correct localhost, don't proceed until status is "OK"

In [2]:
gsi_conf = gsi_float_neural_api.GSIConfig()
print(gsi_conf)
gsi = gsi_float_neural_api.Float_GSI(conf=gsi_conf)

In [5]:
 #gsi.delete_allocation('127b8960-f916-11ea-92f3-0242ac110010')

'OK'

In [6]:
alloc_id, status = gsi.request_allocation(2) #Only 2 right now

print("Allocation ID: " + str(alloc_id))
print("Status: " + str(status))

Allocation ID: 635da5fc-f916-11ea-92f3-0242ac110010
Status: OK


### Load Data

Loading dataset from server since it is mounted there, loading query set from local hdf5 file

In [7]:
dataset_id, result = gsi.import_dataset("/efs/data/qa/any_vision/dataset.npy")
print("Dataset ID: " + str(dataset_id))
print("Status: " + str(result))

Dataset ID: 6426e6c4-f916-11ea-92f3-0242ac110010
Status: OK


In [8]:
number_of_queries = 1000
neural_matrix_path = "/efs/data/public/Daphna/AnyVision_NPHash_Weight_Files/any_vision_1589923812.533218__dim256_loss7_v3random_proxies_nlFalse_hns256.0_scale30_optimizerAdam_lr0.001_proxies5000_trainFalse_hidden2_tss100000000_merged_weights.npy"

In [9]:
#status = gsi.load_data(alloc_id, dataset_id, neural_matrix_path, hamming_k=3000, normalize=False, typical_nqueries=number_of_queries, max_nqueries=number_of_queries)
#print("Status: " +str(status))

### Search

Search using different Hamming values

In [13]:
search = gsi.Search()

hamming_list = [100,150,300,600,1000,1500,3000]
time_list = []

total_search_time = 0
print("Search Times (QPS) for GSI_APU float32 Search:")
print("----------------------------------------------")

for j in range(len(hamming_list)):

    status = gsi.load_data(alloc_id, dataset_id, neural_matrix_path, hamming_k=hamming_list[j], normalize=False, typical_nqueries=1000, max_nqueries=1000)
    
    total_search = 0
    for i in range(10):
        distance, indices, search_time, status = search.knn_composite_cosine(alloc_id, dataset_id, "/efs/data/qa/any_vision/queries_1000.npy")
        time_list.append([distance, indices, search_time])
        total_search = total_search + search_time
        
    time = total_search/10
    print("Hamming_k value of: " + str(hamming_list[j]),end = " runs at ")
    print(str(np.round(1000.0/time,2)) + " Queries per second")
    print()
    
    status = gsi.unload_data(alloc_id, dataset_id)
    

Search Times (QPS) for GSI_APU float32 Search:
----------------------------------------------
Hamming_k value of: 100 runs at 5870.17 Queries per second

Hamming_k value of: 150 runs at 5612.92 Queries per second

Hamming_k value of: 300 runs at 4419.81 Queries per second

Hamming_k value of: 600 runs at 3742.65 Queries per second

Hamming_k value of: 1000 runs at 3244.2 Queries per second

Hamming_k value of: 1500 runs at 2641.84 Queries per second

Hamming_k value of: 3000 runs at 1677.07 Queries per second



In [31]:
status = gsi.unload_data(alloc_id, dataset_id)
status = gsi.delete_allocation(alloc_id)

In [ ]:
attrs = {
        "batch_mode": batch,
        "best_search_time": best_search_time,
        "candidates": avg_candidates,
        "expect_extra": verbose,
        "name": str(algo),
        "run_count": run_count,
        "distance": distance,
        "count": int(count)
    }

In [ ]:
        if batch:
            results = batch_query(X_test)
        else:
            results = [single_query(x) for x in X_test]